In [1]:
import math

import geopandas as gpd
import numpy as np
import pandas as pd
from shapely import LineString, MultiPolygon, Point, Polygon
from shapely.ops import polygonize, unary_union



In [ ]:
import pandas as pd

data = {
    30: {
        63: 0,
        125: 0.0002,
        250: 0.0009,
        500: 0.003,
        1000: 0.0075,
        2000: 0.014,
        4000: 0.025,
        8000: 0.064
    },
    20: {
        63: 0,
        125: 0.0003,
        250: 0.0011,
        500: 0.0028,
        1000: 0.0052,
        2000: 0.0096,
        4000: 0.025,
        8000: 0.083
    },
    10: {
        63: 0,
        125: 0.0004,
        250: 0.001,
        500: 0.002,
        1000: 0.0039,
        2000: 0.01,
        4000: 0.035,
        8000: 0.125
    },
    0: {
        63: 0,
        125: 0.0004,
        250: 0.0008,
        500: 0.0017,
        1000: 0.0049,
        2000: 0.017,
        4000: 0.058,
        8000: 0.156
    }
}

air_resist_ratio = pd.DataFrame(data)


def get_nearest_values(array, value):
    sorted_array = sorted(array)
    if value in sorted_array:
        return [value]
    if value > max(sorted_array):
        return [sorted_array[-1]]
    if value < min(sorted_array):
        return [sorted_array[0]]

    for i, val in enumerate(sorted_array):
        if value < val:
            return sorted_array[max(i - 1, 0)], sorted_array[i]
    return sorted_array[-2], sorted_array[-1]


def get_air_resist_ratio(temp, freq):
    nearest_temp = get_nearest_values(air_resist_ratio.columns, temp)
    nearest_freq = get_nearest_values(air_resist_ratio.index, freq)

    if len(nearest_temp) == 1 and len(nearest_freq) == 1:
        return air_resist_ratio.loc[nearest_freq[0], nearest_temp[0]]

    if len(nearest_temp) == 2 and len(nearest_freq) == 2:
        freq1, freq2 = nearest_freq
        temp1, temp2 = nearest_temp

        coef_temp1_freq1 = air_resist_ratio.loc[freq1, temp1]
        coef_temp1_freq2 = air_resist_ratio.loc[freq2, temp1]
        coef_temp2_freq1 = air_resist_ratio.loc[freq1, temp2]
        coef_temp2_freq2 = air_resist_ratio.loc[freq2, temp2]

        weight_temp1 = (temp2 - temp) / (temp2 - temp1)
        weight_temp2 = (temp - temp1) / (temp2 - temp1)
        weight_freq1 = (freq2 - freq) / (freq2 - freq1)
        weight_freq2 = (freq - freq1) / (freq2 - freq1)

        coef_freq1 = coef_temp1_freq1 * weight_temp1 + coef_temp2_freq1 * weight_temp2
        coef_freq2 = coef_temp1_freq2 * weight_temp1 + coef_temp2_freq2 * weight_temp2

        final_coef = coef_freq1 * weight_freq1 + coef_freq2 * weight_freq2

        return final_coef

    if len(nearest_temp) == 2 and len(nearest_freq) == 1:
        temp1, temp2 = nearest_temp
        freq1 = nearest_freq[0]

        coef_temp1 = air_resist_ratio.loc[freq1, temp1]
        coef_temp2 = air_resist_ratio.loc[freq1, temp2]

        weight_temp1 = (temp2 - temp) / (temp2 - temp1)
        weight_temp2 = (temp - temp1) / (temp2 - temp1)

        return coef_temp1 * weight_temp1 + coef_temp2 * weight_temp2

    if len(nearest_temp) == 1 and len(nearest_freq) == 2:
        temp1 = nearest_temp[0]
        freq1, freq2 = nearest_freq

        coef_freq1 = air_resist_ratio.loc[freq1, temp1]
        coef_freq2 = air_resist_ratio.loc[freq2, temp1]

        weight_freq1 = (freq2 - freq) / (freq2 - freq1)
        weight_freq2 = (freq - freq1) / (freq2 - freq1)

        return coef_freq1 * weight_freq1 + coef_freq2 * weight_freq2


In [ ]:
import numpy as np
from scipy.optimize import fsolve


def dist_to_target_db(init_noise_db, target_noise_db, geometric_mean_freq_hz, air_temperature):
    def equation(r):
        return L - L_ist + 20 * np.log10(r) + k * r

    L_ist = init_noise_db
    L = target_noise_db
    k = get_air_resist_ratio(air_temperature, geometric_mean_freq_hz)
    initial_guess = 1
    r_solution = fsolve(equation, initial_guess)

    # print(
    #     f"Шум громкостью {init_noise_db} дБ,среднегеометрической частотой {geometric_mean_freq_hz} Гц при температуре {air_temperature} Цельсия затухает к шуму {target_noise_db} дБ за расстояние {r_solution} \nКоэффициент сопротивления воздуха {k}")

    return r_solution[0]


dist_to_target_db(100, 40, 1000, 20)

In [ ]:
from shapely import Point
import geopandas as gpd
from objectnat import get_visibility_accurate

def simulate_noise(point_from: Point, obstacles: gpd.GeoDataFrame, dB_step=5, **kwargs):
    init_noise_db = kwargs['init_noise_db']
    target_noise_db = kwargs['target_noise_db']
    geometric_mean_freq_hz = kwargs['geometric_mean_freq_hz']
    air_temperature = kwargs['air_temperature']
    
    absorb_ratio_column = kwargs.get('absorb_ratio',None)
    if absorb_ratio_column is None:
        obstacles['absorb_ratio'] = 0.03
    
    donuts = []
    values = []
    dist = 0
    
    cur_dB = init_noise_db - dB_step
    to_cut_off = point_from
    while cur_dB != target_noise_db-dB_step:
        dist = dist_to_target_db(init_noise_db, cur_dB, geometric_mean_freq_hz, air_temperature)
        cur_buffer = point_from.buffer(dist)
        donuts.append(cur_buffer.difference(to_cut_off))
        values.append(cur_dB)
        to_cut_off = cur_buffer
        cur_dB = cur_dB - dB_step

    vis_poly = get_visibility_accurate(point_from,obstacles,dist)
    noise_df = gpd.GeoDataFrame(data={'noise_dB':values},geometry=donuts,crs=32636)
    return vis_poly
    return noise_df.clip(vis_poly)


start_p = Point(348065, 6647674)
obstacles = gpd.read_parquet('buildings.parquet').to_crs(epsg=32636)
res = simulate_noise(start_p,
               obstacles,
               init_noise_db= 100,
               target_noise_db=40,
               geometric_mean_freq_hz=1000,
               air_temperature=20,
               )


In [ ]:
m1= gpd.GeoDataFrame(geometry=[res],crs=32636).explore()
gpd.GeoDataFrame(geometry=[Point(coords) for coords in res.simplify(0.1).exterior.coords],crs=32636).explore(m=m1,color='red')
gpd.GeoDataFrame(geometry=[vis_poly2],crs=32636).explore(m=m1)
obstacles.explore(m=m1,color='pink')

In [ ]:
%reload_ext autoreload
%autoreload 2

new_obs = pd.concat([obstacles,gpd.GeoDataFrame(geometry=[res.buffer(-0.1)],crs=32636)])
p = [Point(coords) for coords in res.simplify(1).exterior.coords][14]
vis_poly2 = get_visibility_accurate(p,new_obs,500)
# m1=new_obs.explore(color='red')
# m1= gpd.GeoDataFrame(geometry=[vis_poly],crs=32636).explore()
# gpd.GeoDataFrame(geometry=[Point((347810.97481683653,6647261.617306325))],crs=32636).explore(m=m1)

In [ ]:
gpd.GeoDataFrame(geometry=vis_poly,crs=32636)

In [ ]:
m1 = res.explore(column='noise_dB',cmap='RdYlGn_r',vmin=10,tiles='CartoDB positron')
obstacles.explore(m=m1,color='pink')
m1

In [ ]:
m1 = obstacles.explore(color='pink')
gpd.GeoDataFrame(geometry=[result2], crs=32636).explore(m=m1)
gpd.GeoDataFrame(geometry=[Point(coords) for coords in result2.exterior.coords], crs=32636).explore(m=m1, color='red')

points = [Point(coords) for coords in result2.simplify(1).exterior.coords]
print(len(points))
gpd.GeoDataFrame(geometry=points, crs=32636).explore(m=m1, color='purple')
